# Exercises

In these exercises, we will look at stochastic simulation to model single cells. We will compare with modelling that uses ordinary differential equations and try to understand any differences.

### Question 1
We begin by studying a model of gene expression:

> $\emptyset \xrightarrow[]{u} \text{mRNA}$ 

> $\text{mRNA} \xrightarrow[]{d_0}  \emptyset$

> $\text{mRNA} \xrightarrow[]{v} \text{mRNA} + \text{protein}$ 

> $\text{protein} \xrightarrow[]{d_1}  \emptyset$

These equations model constitutive expression.

1. Write down and solve the differential equations for this model using ${\tt odeint}$. Use $u= 1 \, {\rm s}^{-1}$, $v= 1 \, {\rm s}^{-1}$, $d_0= 0.1 \, {\rm s}^{-1}$ and $d_1= 0.1 \, {\rm s}^{-1}$ and 100 seconds of simulation. Start with zero mRNAs and zero proteins.
<br><br>

2. Using the model file for gene expression provided and the StochPy package, simulate 5 single-cell time-series starting from zero mRNAs. Plot these time-series using the built-in functions. 
<br><br>
The code should look something like:

    ```python
import stochpy 
smod= stochpy.SSA()
# load a model
smod.Model('model file', dir= 'model directory')
smod.DoStochSim(end= end_time, mode= 'time', 
    trajectories= number_of_trajectories, 
    quiet= False)
smod.PlotSpeciesTimeSeries()
    ```

3. We will investigate the mean and variance of these time-series. Increase the number of trajectories to 100 (which may take some time and you might want to start with 10 until your code works). Using the function $\texttt{extract}$_$\texttt{data}$ (given below), extract the trajectories as a numpy array. Calculate the mean across the population of cells at each time point. Plot both the mean and your result from the differential equations on the same graph.
<br><br>
4. The Fano factor is sometimes used to measure noise and is defined as the variance of a distribution divided by the mean of the distribution. 
<br><br>
Using 
```
smod.ChangeParameter('u', new_parameter_value)
smod.ChangeParameter('v', new_parameter_value)
```
to change the synthesis rates and a $\texttt{for}$ loop, re-run the simulation for the parameter pairs: $(u= 10 \, {\rm s}^{-1}, v= 0.1 \, {\rm s}^{-1})$, $(u= 1 \, {\rm s}^{-1}, v= 1 \, {\rm s}^{-1})$ and $(u= 1 \, {\rm s}^{-1}, v= 10 \, {\rm s}^{-1})$. Process the data using $\texttt{extract}$_$\texttt{data}$. 
<br><br>
For each pair of parameters, plot the mean and the Fano factor over the 100 trajectories at each time point for the last 50 time points. 
<br><br>

5. Rerun the code for question 4. The results change substantially between runs because of the low number of trajectories. 
<br><br>
For clearer results, you have been provided with a pickle file of 10,000 runs. Repeat the plotting from question 4 using these 10,000 runs. What do you notice about the Fano factors for protein and mRNA? Looking at a few of the single-cell time-series, can you understand why lower rate of mRNA synthesis give a higher Fano factor for proteins?
<br><br>

6. An alternative measure of variation is the [coefficient of variation](https://en.wikipedia.org/wiki/Coefficient_of_variation), which is defined as the the standard deviation of a distribution divided by its mean. 
<br><br>
Calculate the coefficient of variation for the final time point of the mRNA time-series for the three parameter sets used in question 4. Plot the coefficient of variation against the mean for the final time point for the three parameter sets. What do you notice, and what does this imply for the variation in mRNA as mean expression increases? Think about the difference between Fano Factor and coefficient of variation. Which do you think is a better measure of the noise in a population of cells?

In [ ]:
def extract_data(smod, ntps= 100):
    '''
    extracts trajectories onto regularly spaced time points
    
    Returns
    --
    gridded_data: a list of numpy arrays, one for each species where 
        each column corresponds to a trajectory
    t: the corresponding time points 
    species: a list of the chemical species simulated
    '''
    smod.GetRegularGrid(ntps)
    species= smod.data_stochsim_grid.species_labels
    ntraj= smod.sim_trajectories_done
    t= smod.data_stochsim_grid.getTime()
    # grid data
    gridded_data= [np.transpose([smod.data_stochsim_grid.species[i][j] 
                             for j in range(ntraj)]) 
               for i in range(len(species))]
    return gridded_data, t, species

## Question 2

We will now consider the repressilator (which you will remember from [assignment 2](http://swainlab.bio.ed.ac.uk/psb/assignment2/assignment2.html)).

1. As in assignment 2, simulate this system using ${\tt odeint}$ with $a = 100 \, {\rm s}^{-1}$ and $b= 0.1 \, {\rm s}^{-1}$. Convince yourself that the system will oscillate by running the simulation for 1000 seconds. 
<br><br>
2. Write a StochPy model file for the same model. Simulate 10 traces for 1000s and plot the mean. How does it compare to the solution using differential equations? Confirm your results using the 10,000 traces provided in the pickle file. Looking at a few traces, can you gain some intuition as to what is happening? What does this imply for genetic oscillators in multicellular organisms?

## Question 3

Why do we use differential equations at all? Why not use stochastic simulation for everything?

***
### Loading data for questions

In [14]:
import pickle
dd= pickle.load(open("pkls/gene_expression_10000.pkl", "rb"))
t= dd['timepoints']
data= dd['data']
params= dd['parameters']

The object ${\tt dd}$ is a dictionary: ${\tt parameters}$ is a list of the values for the $(u, v)$ parameters: ${\tt data}$ is a list with each entry in the list containing the 10,000 trajectories for the corresponding parameters in ${\tt parameters}$. In fact, each entry in ${\tt data}$ is itself a list, with the first array being the mRNA trajectories and the 2nd array being the corresponding protein trajectories (each [100 x 10000] in size).

In [16]:
rd= pickle.load(open("pkls/repressillator_10000.pkl", "rb"))
t= rd['timepoints']
data= rd['data']
params= rd['parameters']